# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [5]:
import numpy as np
import pandas as pd

In [74]:
pd.set_option('display.max_colwidth', None)

In [3]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"], 
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [1543, 3345, 533, 11, 43],
                                      [156, 3345, 10, 15, 1134]
                                  ],
             "bought_list": [
                             [156,27],
                             [11,43],
                             [1]
                            ]})

In [4]:
recommended_list = [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [6]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list[:k], bought_list)
    
    recall = flags.sum() / bought_list.shape[0]
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list[:k], bought_list)
    recall = (flags * prices_recommended).sum() / prices_bought.sum()
    
    return recall

In [7]:
data.apply(lambda x: recall_at_k(x["recommended_list"], x["bought_list"], 3), 1).mean()

0.16666666666666666

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [8]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    ranks = np.arange(k)[np.isin(recommended_list[:k], bought_list)]
    
    if not ranks.any(): 
        MMR = 1 / min_rank.min()
    else:
        MMR = None
    
    return MMR

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [79]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list[:k], bought_list) / np.log2(np.arange(1,k+1) + 1)
    DCG = flags.sum()
    
    if k < bought_list.shape[0]:
        i_flags = np.isin(recommended_list[:k], bought_list) / \
        np.log2(np.arange(1,k+1) + 1)
    else:
        i_flags = np.isin(recommended_list[:bought_list.shape[0]], bought_list)  / \
        np.log2(np.arange(1,bought_list.shape[0]+1) + 1)
    IDCG = i_flags.sum()
    if IDCG == 0:
        NDCG = 0
    else:
        NDCG = DCG / IDCG
    
    return NDCG    

In [80]:
data.apply(lambda x: ndcg_at_k(x["recommended_list"], x["bought_list"], 3), 1).mean()

0.6666666666666666

In [81]:
data.apply(lambda x: ndcg_at_k(x["recommended_list"], x["bought_list"], 5), 1)

0    1.613147
1    1.257902
2    0.000000
dtype: float64